In [1]:
import csv, pickle, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from autosklearn.regression import AutoSklearnRegressor

from sklearn.linear_model import LinearRegression
from fbprophet import Prophet
from pmdarima.arima import auto_arima
from keras.models import Sequential
from keras.layers import LSTM, Dense


/root/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-02 21:12:13.012649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 21:12:13.872109: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
buildings_list = ['Energy_Innovation_Center_Data.csv', 'Stadium_Data.csv']

in_path = './clean_data/'

'''
Methods for preprocessing data by filling in missing gaps and generating synthetic data:
1. linear regression
2. linear interpolation, 
3. cubic interpolation (through a coarse sampling of the data, perhaps, including a few months before and a few months after the large data gap) -- although I think this probably will not change the results much. 
4. Prophet. 
5. Others. 
6. Another way to fill the data could be to use a data "mosaic" technique, by looking though the data from the previous years, and by "pattern-match" the data gap somehow and copy the old data over to fill the gap. Some "transformation and scaling of the old data might be necessary"
'''

models = {}

# types of models
model_types = ['ensembles', 'solos']

# types of preprocessing methods
# preprocessing_methods = ['Linear_Regression', 'Linear_Interpolation', 'Prophet']
preprocessing_methods = ['Linear_Regression']

time_steps = [1, 8, 12, 24]

for model_type in model_types:
    out_path = f'./models/{model_type}/'

    for building in buildings_list:
        df = pd.read_csv(in_path + building)

        # Convert the data into a Pandas dataframe
        df['ts'] = pd.to_datetime(df['ts'])
        df = df.drop_duplicates(subset=['bldgname', 'ts'])
        df = df.sort_values(['bldgname', 'ts'])

        # Group the dataframe by building name and timestamp
        groups = df.groupby('bldgname')
        df = df.set_index('ts')

        orig_cols = df.columns
        y_columns = ['present_elec_kwh', 'present_htwt_mmbtu', 'present_wtr_usgal', 'present_chll_tonhr', 'present_co2_tons']
        header = ['ts'] + y_columns

        print(building)

        for name, group in groups:
            bldgname = name

            group = group.drop_duplicates(subset=['ts'])

            for y in y_columns:
                col_data = group[header]
                if col_data[y].count() >= 365*24 and y != 'present_co2_tons':

                    for preprocessing_method in preprocessing_methods:
                        model_data = col_data.copy()
                        model_data = model_data.rename(columns={ y: 'y', 'ts': 'ds' })
                        model_data = model_data.sort_values(['ds'])

                        # save the original values into new column
                        model_data['y_saved'] = model_data['y']


                        # *** Linear Regression (1/5) ***
                        if (preprocessing_method == 'Linear_Regression'):
                            m = LinearRegression()

                            X_train = model_data[model_data['y'].notna()]['ds'].values.reshape(-1, 1)
                            y_train = model_data[model_data['y'].notna()]['y'].values.reshape(-1, 1)
                            m.fit(X_train, y_train)

                            X_test = model_data[model_data['y'].isna()]['ds'].values.reshape(-1, 1)

                            X_test = X_test.astype(np.float32)

                            y_pred = m.predict(X_test)

                            model_data.loc[model_data['y'].isna(), 'y'] = y_pred.flatten()

                        # *** Linear Interpolation (2/5) ***
                        elif(preprocessing_method == 'Linear_Interpolation'):
                            model_data['y'] = model_data['y'].interpolate(method='linear', limit_direction='both')    
                        

                        # *** Cubic Interpolation (3/5) ***
                        elif(preprocessing_method == 'Cubic_Interpolation'):
                            model_data['y'] = model_data['y'].interpolate(method='cubic', limit_direction='both')


                        # *** Prophet (4/5) ***
                        elif(preprocessing_method == 'Prophet'):
                            m = Prophet()
                            m.fit(model_data)
                            future = m.make_future_dataframe(periods=0, freq='H')
                            forecast = m.predict(future)
                            model_data['y'] = model_data['y'].fillna(forecast['yhat'])

                        # *** ARIMA (daily seasonality, m=24) (5/5) ***
                        elif(preprocessing_method == 'ARIMA'):
                            m = auto_arima(model_data[model_data['y'].notna()]['y'].values, seasonal=True, m=24, suppress_warnings=True)

                            y_pred, conf_int = m.predict(n_periods=model_data['y'].isna().sum(), return_conf_int=True)
                            model_data.loc[model_data['y'].isna(), 'y'] = y_pred
                        else:
                            sys.exit(0)


                        # normalize the data, save orginal data column for graphing later
                        scaler = MinMaxScaler(feature_range=(0, 1))
                        data_scaled = scaler.fit_transform(model_data['y'].values.reshape(-1, 1))
                        saved_data_scaled = scaler.fit_transform(model_data['y_saved'].values.reshape(-1, 1))

                        # split the data into training and testing sets
                        train_size = int(len(data_scaled) * 0.8)
                        test_size = len(data_scaled) - train_size
                        train_data = data_scaled[0:train_size,:]
                        test_data = data_scaled[train_size:len(data_scaled),:]
                        saved_test_data = saved_data_scaled[train_size:len(data_scaled),:]

                        for time_step in time_steps:
                            # define the window size
                            window_size = time_step

                            # create the training data set
                            def create_dataset(dataset, window_size):
                                X, y = [], []
                                for i in range(window_size, len(dataset)):
                                    X.append(dataset[i-window_size:i, 0])
                                    y.append(dataset[i, 0])
                                X, y = np.array(X), np.array(y)
                                return X, y

                            X_train, y_train = create_dataset(train_data, window_size)

                            # create the testing data set
                            X_test, y_test = create_dataset(test_data, window_size)
                            saved_X_test, saved_y_test = create_dataset(saved_test_data, window_size)

                            # reshape the input data
                            X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
                            X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))
                            saved_X_test = np.reshape(saved_X_test, (saved_X_test.shape[0], saved_X_test.shape[1]))

                            # 2 hours each task
                            time_dist = 60*1

                            # Create the model (solo or ensemble)
                            if model_type == 'solos':
                                model = AutoSklearnRegressor(
                                    time_left_for_this_task=time_dist,
                                    per_run_time_limit=int(time_dist/10),
                                    memory_limit = 102400,
                                    ensemble_kwargs = {'ensemble_size': 1}
                                )
                            else:
                                model = AutoSklearnRegressor(
                                    time_left_for_this_task=time_dist,
                                    per_run_time_limit=int(time_dist/10),
                                    memory_limit = 102400,
                                )


                            # # Train the model
                            model.fit(X_train, y_train)
                            
                            # # Predict on the test set
                            y_pred = model.predict(X_test)

                            # # Inverse transform the predictions and actual values
                            y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
                            y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
                            saved_y_test = scaler.inverse_transform(saved_y_test.reshape(-1, 1))
                            y_train = scaler.inverse_transform(y_train.reshape(-1, 1))

                            # save the model
                            model_title = f'{bldgname}_{y}_{preprocessing_method}_{time_step}_model'
                            model_file = f'{out_path}{model_title}'

                            # with open(model_file + '.pkl', 'wb') as file:
                            #     pickle.dump(model, file)

                            # calculate metrics
                            print(f'{bldgname}, {y}')
                            print(test_size, len(y_test))
                            print(model.leaderboard())

                            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                            print('RMSE: %.3f' % rmse)

                            mae = mean_absolute_error(y_test, y_pred)
                            print('MAE: %.3f' % mae)

                            r2 = r2_score(y_test, y_pred)
                            print('R2: %.3f' % r2)

                            # # save results
                            models[(model_type, bldgname, y, preprocessing_method, time_step)] = (rmse, mae, r2, model_file)

                            # plot results
                            fig, ax = plt.subplots()

                            # Plot the actual values
                            # ax.plot(np.concatenate([y_train, y_test]), label='Actual Values')
                            ax.plot(y_test, label='Actual Values', alpha=0.7)

                            # # Plot the predictions
                            # ax.plot(range(train_len, train_len + len(y_test)), y_pred, label='Predicted Values')
                            ax.plot(y_pred, label='Forecasted Values', alpha=0.8)

                            # # Plot the replaced missing values
                            nan_mask = np.isnan(saved_y_test)  # boolean mask of NaN values in saved_y_test
                            y_test[~nan_mask] = np.nan
                            
                            ax.plot(y_test, label='Predicted Values', alpha=0.75)

                            ax.set_title(f'{bldgname}_{preprocessing_method} Consumption')
                            ax.set_xlabel('Time (Hours)')
                            ax.set_ylabel(y.split('_')[-2] + ' (' + y.split('_')[-1] + ')')

                            ax.legend()
                            plt.grid(True)
                            plt.savefig(model_file + '.png')
                            plt.close(fig)

Energy_Innovation_Center_Data.csv
Energy Innovation Center, present_elec_kwh
5933 5932
          rank  ensemble_weight            type      cost  duration
model_id                                                           
5            1             0.40     extra_trees  0.413554  1.537498
8            2             0.52     extra_trees  0.413555  1.567113
4            3             0.08  ard_regression  0.435970  0.352086
RMSE: 7.726
MAE: 5.520
R2: 0.287
[WARNING] [2023-05-02 21:13:19,035:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-05-02 21:13:26,083:Client-EnsembleBuilder] No runs were available to build an ensemble from
Energy Innovation Center, present_elec_kwh
5933 5921
          rank  ensemble_weight                 type      cost  duration
model_id                                                                
5            1             0.78    gradient_boosting  0.252278  3.093846
4            2             0.04       ard_regression

/root/miniconda3/envs/tf/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -126.586
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       69342.1    0.00820794       5085.66           1           1      119   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       69578.9    0.00356649       1115.32           1           1      233   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       69637.4    0.00289759       2175.19           1           1      341   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       69759.1     0.0119279       3502.12           1           1      458   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       69801.2    0.00497302       1821.24           1           1      571   
    Iter      log prob        ||dx||      ||grad||       alpha  

/root/miniconda3/envs/tf/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/root/miniconda3/envs/tf/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Energy Innovation Center, present_elec_kwh
5933 5932
          rank  ensemble_weight                 type      cost  duration
model_id                                                                
5            1             0.34          extra_trees  0.396673  1.770873
20           2             0.44        random_forest  0.396847  2.135167
18           3             0.08          extra_trees  0.398608  2.024012
4            4             0.12       ard_regression  0.408919  0.341953
10           5             0.02  k_nearest_neighbors  0.638141  0.395772
RMSE: 7.718
MAE: 5.430
R2: 0.319
[WARNING] [2023-05-02 21:19:37,889:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-05-02 21:19:44,945:Client-EnsembleBuilder] No runs were available to build an ensemble from
Energy Innovation Center, present_elec_kwh
5933 5921
          rank  ensemble_weight                 type      cost  duration
model_id                                                     

...

Energy Innovation Center, present_chll_tonhr
5933 5932
          rank  ensemble_weight               type      cost  duration
model_id                                                              
20           1             0.54      random_forest  0.206039  2.353858
7            2             0.28  gradient_boosting  0.207628  1.602663
5            3             0.18        extra_trees  0.210335  1.564291
RMSE: 0.114
MAE: 0.090
R2: 0.436
[WARNING] [2023-05-02 21:37:46,449:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-05-02 21:37:53,524:Client-EnsembleBuilder] No runs were available to build an ensemble from
Energy Innovation Center, present_chll_tonhr
5933 5921
          rank  ensemble_weight                 type      cost  duration
model_id                                                                
5            1             0.64    gradient_boosting  0.153983  3.168747
4            2             0.30       ard_regression  0.162379  0.4

/root/miniconda3/envs/tf/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -1044.64
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       46776.6     0.0207772       1050.22           1           1      121   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       46838.2    0.00706483       749.978      0.7375      0.7375      233   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       46895.9     0.0151457       2262.51           1           1      342   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       46936.3    0.00302904       725.702           1           1      454   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       46957.7    0.00295037       203.191           1           1      563   
    Iter      log prob        ||dx||      ||grad||       alpha  

/root/miniconda3/envs/tf/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/root/miniconda3/envs/tf/lib/python3.9/site-packages/fbprophet/forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Stadium, present_elec_kwh
5933 5932
          rank  ensemble_weight               type      cost  duration
model_id                                                              
7            1             0.48  gradient_boosting  0.272107  2.292070
5            2             0.10        extra_trees  0.273653  2.050264
4            3             0.34     ard_regression  0.275018  0.377042
21           4             0.08      random_forest  0.279816  2.957617
RMSE: 39.142
MAE: 29.408
R2: 0.620
[WARNING] [2023-05-02 21:46:58,097:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-05-02 21:47:05,186:Client-EnsembleBuilder] No runs were available to build an ensemble from
Stadium, present_elec_kwh
5933 5921
          rank  ensemble_weight                 type      cost  duration
model_id                                                                
5            1             0.64    gradient_boosting  0.125771  3.633978
4            2             0.14 

In [ ]:
# Create a CSV files to save the results

header = ['model_type','bldgname', 'y', 'preprocessing_method', 'time_step', 'rmse', 'mae', 'r2', 'model_file']
rows = []

# create csv file for each model folder
for m_type in model_types:
    out_path = f'./models/{m_type}/'

    with open(f'{out_path}results.csv', mode='w') as results_file:
        writer = csv.writer(results_file)
        writer.writerow(header)

        # Plot the actual values and predictions
        for name, (rmse, mae, r2, model_file) in models.items():
            model_type, bldgname, y, preprocessing_method, time_step = name

            # Write the row to the CSV file
            if m_type == model_type:
                row = [model_type, bldgname, y, preprocessing_method, time_step, rmse, mae, r2, model_file + '.pkl']
                writer.writerow(row)
                rows.append(row)

# create master results csv
with open('results.csv', mode='w') as results_file:
    writer = csv.writer(results_file)
    writer.writerow(header)

    for row in rows:
        writer.writerow(row)